# 简介 

**之前忘记了ffmpeg在Python里面的使用方法，所以用了比较复杂的给视频调帧率和加音轨的方法，这里感谢[@drrr_us](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/105806)**大佬做出的修改，将提升转换帧率及添加音轨的效率
项目地址:[点我](https://aistudio.baidu.com/aistudio/projectdetail/790567)

**嘛...搞出来自己玩玩或者给朋友发就行了...还是不要传到b站为好**

>   原作者:[我](https://github.com/AliaksandrSiarohin/first-order-model)还有[我](https://github.com/dunnousername/yanderifier)

教程:[并不官方的教程](https://www.bilibili.com/video/BV1Mv41117LV)

提示:本数据集为"换脸"数据集,全部的数据集可以在[这里](https://yadi.sk/d/lEw8uRm140L_eQ)找到

1.vox-cpk.pth.tar效果[点我](https://gitee.com/xiaoxu12315/first-order-model/raw/master/sup-mat/vox-teaser.gif)

2.mgif-cpk.pth.rar效果[点我](https://gitee.com/xiaoxu12315/first-order-model/raw/master/sup-mat/mgif-teaser.gif)

3.fashion.pth.tar效果[点我](https://gitee.com/xiaoxu12315/first-order-model/raw/master/sup-mat/fashion-teaser.gif)

# 其它玩法

**我已经集成了这三个数据集,可以简单使用.**
![](https://ai-studio-static-online.cdn.bcebos.com/355f06668b8a4ead88e684c952aaa4239e09e666253c4f1c91b931e59946606c)
```
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/vox-256.yaml', 
                            checkpoint_path='data/checkpoint.tar')
                            #加载配置文件并加载数据集
```
**换动作视频修改为**
```
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/fashion-256.yaml', 
                            checkpoint_path='data/fashion.pth.tar')
                            #加载配置文件并加载数据集
```
**换gif视频修改为**
```
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/mgif-256.yaml', 
                            checkpoint_path='data/mgif-cpk.pth.tar')
                            #加载配置文件并加载数据集
```


 **在file文件夹内放素材,将视频更改为import.mp4,图片更改为import.png**

**视频和图片推荐分辨率:256x256,视频时长:一分钟内**

**百度AI Studio不支持TF框架,可能会弹出不支持TF框架的提示,重启项目便可**
表现:![](https://ai-studio-static-online.cdn.bcebos.com/a4ff3640d9a6486d99cf2749c27e8b681ac7bbf770644b94b692346eb0a74067)


# 代码

In [1]:
!pwd

/home/chy/Yanderifier


In [2]:
#查看数据集
!ls /home/chy/Yanderifier/data

checkpoint.tar	fashion.pth.tar  mgif-cpk.pth.tar


In [3]:
!ls /home/chy/Yanderifier/work

In [4]:
import sys
sys.path.append('/home/chy/Yanderifier/external-libraries')
sys.path.append('/home/chy/Yanderifier/first-order-model')

In [5]:
#!pip3 install --user wheel

In [6]:
#!git clone https://gitee.com/xiaoxu12315/first-order-model.git
#从码云克隆仓库
!cd /home/chy/Yanderifier/first-order-model
#进入工作目录
#!pip3 install --user -i https://mirrors.aliyun.com/pypi/simple/ moviepy cffi==1.11.5 cloudpickle==1.2.1 cycler==0.10.0 dask==0.18.2 decorator==4.3.0 imageio==2.5.0 kiwisolver==1.0.1 matplotlib==2.2.2 networkx==2.1 numpy==1.15.0 pandas==0.23.4 Pillow==5.2.0 pycparser==2.18 pygit==0.1 pyparsing==2.2.0 python-dateutil==2.7.3 pytz==2018.5 PyWavelets==0.5.2 PyYAML==5.1 scikit-image==0.14.0 scikit-learn==0.19.2 scipy==1.1.0 six==1.14.0 toolz==0.9.0 torch==1.0.0 torchvision==0.2.1 tqdm==4.24.0
#安装依赖

In [7]:
#!pip3 install --user imageio_ffmpeg

In [8]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('file/import.png')
driving_video = imageio.mimread('file/import.mp4')


#调整图片和视频分辨率

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

In [9]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/vox-256.yaml', 
                            checkpoint_path='data/checkpoint.tar')
                            #加载配置文件并加载数据集

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#保存结果
imageio.mimsave('file/result-10fps.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display(source_image, driving_video, predictions).to_html5_video())

 17%|█▋        | 192/1152 [00:05<00:25, 37.13it/s]

In [ ]:
import cv2
from cv2 import VideoWriter,VideoWriter_fourcc
!pwd
video_root ='file/import.mp4'
cap = cv2.VideoCapture(video_root)
print (cap)
fps = cap.get(cv2.CAP_PROP_FPS)
print (fps)
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
resize_width = 256
resize_height = 256
double_fps = 2*fps
print(video_height)
print(video_width)
print(fps)

In [ ]:
import cv2
from cv2 import VideoWriter,VideoWriter_fourcc

video_root ="file/result-10fps.mp4"
out_root = "file/result-origin-fps.mp4"
cap = cv2.VideoCapture(video_root)
resize_width = 256
resize_height = 256
double_fps = 2*fps

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
videoWriter = cv2.VideoWriter(out_root,fourcc,fps,(256,256))

success = cap.isOpened()
while (success):
    success,frame = cap.read()
    print("Read a new frame:",success)
    if not success:
        break
    frame = cv2.resize(frame,(256,256))
    videoWriter.write(frame)
videoWriter.release()

In [ ]:
from moviepy.editor import *

# 读取2个视频文件 
videoclip_1 = VideoFileClip("file/import.mp4")
videoclip_2 = VideoFileClip("file/result-origin-fps.mp4")

# 提取第一个视频文件的音频部分
audio_1 = videoclip_1.audio

# 将提取的音频和第二个视频文件进行合成
videoclip_3 = videoclip_2.set_audio(audio_1)

# 输出新的视频文件
videoclip_3.write_videofile("file/export.mp4", audio_codec="aac")